In [6]:
# ==========================================
# 2025년 최신 LlamaIndex ReAct Agent RAG 시스템
# ==========================================

# 필요한 패키지 설치 명령어
# pip install llama-index llama-index-llms-openai llama-index-embeddings-openai python-dotenv

import os
from pathlib import Path
from dotenv import load_dotenv

# 환경변수 로드 (.env 파일에서 API 키 읽기)
load_dotenv()

# OpenAI API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY를 .env 파일이나 환경변수에 설정해주세요.")

# 설치된 패키지 버전 확인 (선택적)
try:
    import importlib.metadata as metadata
    print(f"llama-index 버전: {metadata.version('llama-index')}")
    print(f"llama-index-core 버전: {metadata.version('llama-index-core')}")
    print("-" * 50)
except ImportError:
    print("패키지 버전 정보를 확인할 수 없습니다.")

# LlamaIndex 핵심 모듈 임포트
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI  # OpenAI LLM
from llama_index.embeddings.openai import OpenAIEmbedding  # OpenAI 임베딩
from llama_index.core.tools import QueryEngineTool, ToolMetadata  # 도구 정의
from llama_index.core.agent import ReActAgent  # ReAct 에이전트

# ==========================================
# 1. LLM 및 임베딩 모델 설정
# ==========================================

# GPT-4o-mini를 기본 LLM으로 설정 (비용 효율적이면서 성능 좋음)
Settings.llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.1  # 일관된 응답을 위해 낮은 temperature 설정
)

# OpenAI의 최신 임베딩 모델 사용 (text-embedding-3-large)
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-large",
    dimensions=1024  # 차원 수 명시적 설정
)

print("LLM 및 임베딩 모델 설정 완료")

# ==========================================
# 2. 데이터 디렉토리 설정 및 샘플 데이터 생성
# ==========================================

# 데이터 저장 디렉토리 생성
DATA_DIR = Path("./data/documents")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# 샘플 정책 문서가 없으면 생성
sample_files = {
    "ai_policy_2025.txt": """
    2025년 AI 정책 가이드라인
    
    1. 핵심 원칙
    - 인간 중심의 AI 개발
    - 투명성과 설명 가능성 보장
    - 개인정보보호 및 프라이버시 강화
    - 차별 방지 및 공정성 확보
    
    2. 기술적 요구사항
    - 차등 프라이버시(Differential Privacy) 적용 권장
    - AI 모델의 편향성 정기 검사 의무화
    - 데이터 최소화 원칙 준수
    
    3. 거버넌스
    - AI 윤리 위원회 설치 의무
    - 분기별 영향 평가 실시
    - 시민 참여 메커니즘 구축
    """,
    
    "privacy_guidelines.txt": """
    개인정보보호 가이드라인 2025
    
    1. 데이터 수집 원칙
    - 목적 제한 원칙: 명시된 목적으로만 수집
    - 최소 수집 원칙: 필요한 최소한의 데이터만 수집
    - 동의 기반 수집: 명확한 동의 절차 필수
    
    2. 데이터 보관 및 처리
    - 암호화 저장 의무
    - 접근 권한 최소화
    - 정기적 데이터 삭제
    
    3. 개인의 권리
    - 데이터 열람권
    - 수정 및 삭제 요구권
    - 처리 중단 요구권
    """
}

# 샘플 파일이 없으면 생성
for filename, content in sample_files.items():
    filepath = DATA_DIR / filename
    if not filepath.exists():
        filepath.write_text(content.strip(), encoding="utf-8")
        print(f"샘플 파일 생성: {filename}")

print("데이터 디렉토리 및 샘플 데이터 준비 완료")

# ==========================================
# 3. 문서 로딩 및 벡터 인덱스 생성
# ==========================================

# SimpleDirectoryReader를 사용하여 모든 텍스트 파일 로드
print("문서 로딩 중...")
documents = SimpleDirectoryReader(
    input_dir=str(DATA_DIR),
    recursive=True,  # 하위 디렉토리도 검색
    required_exts=[".txt"]  # .txt 파일만 처리
).load_data()

print(f"로드된 문서 수: {len(documents)}")

# 문서들로부터 벡터 인덱스 생성
print("벡터 인덱스 생성 중...")
vector_index = VectorStoreIndex.from_documents(
    documents,
    show_progress=True  # 진행률 표시
)

print("벡터 인덱스 생성 완료")

# ==========================================
# 4. 쿼리 엔진 및 도구 생성
# ==========================================

# 벡터 인덱스로부터 쿼리 엔진 생성
query_engine = vector_index.as_query_engine(
    similarity_top_k=5,  # 상위 5개 관련 문서 검색
    response_mode="compact",  # 간결한 응답 모드
    verbose=True  # 검색 과정 표시
)

# 쿼리 엔진을 도구로 래핑
document_search_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="document_search",
        description=(
            "AI 정책, 개인정보보호 가이드라인 등 정책 문서를 검색하는 도구입니다. "
            "정책 관련 질문이나 규정에 대한 정보가 필요할 때 사용하세요."
        )
    )
)

print("쿼리 엔진 및 검색 도구 생성 완료")

# ==========================================
# 5. ReAct 에이전트 생성
# ==========================================

# 2025년 최신 방식으로 ReAct 에이전트 생성
try:
    react_agent = ReActAgent.from_tools(
        tools=[document_search_tool],  # 사용할 도구 리스트
        llm=Settings.llm,  # 설정된 LLM 사용
        verbose=True,  # 추론 과정 출력
        max_iterations=5,  # 최대 반복 횟수
        allow_parallel_tool_calls=False  # 순차적 도구 호출
    )
    print("ReAct 에이전트 생성 완료")
    
except AttributeError as e:
    print(f"ReAct 에이전트 생성 실패: {e}")
    print("대안: 기본 쿼리 엔진을 직접 사용합니다.")
    react_agent = None

# ==========================================
# 6. 에이전트 테스트 함수
# ==========================================

def test_agent_query(query: str):
    """에이전트 또는 쿼리 엔진을 사용하여 질의응답 수행"""
    print(f"\n질문: {query}")
    print("=" * 60)
    
    try:
        if react_agent:
            # ReAct 에이전트 사용
            response = react_agent.chat(query)
            print("\n답변:")
            print("-" * 40)
            print(str(response))
        else:
            # 쿼리 엔진 직접 사용 (폴백)
            response = query_engine.query(query)
            print("\n답변:")
            print("-" * 40)
            print(str(response))
            
    except Exception as e:
        print(f"오류 발생: {e}")

# ==========================================
# 7. 실제 테스트 실행
# ==========================================

if __name__ == "__main__":
    # 테스트 질문들
    test_queries = [
        "2025년 AI 정책의 핵심 원칙은 무엇인가요?",
        "차등 프라이버시에 대해 설명해주세요.",
        "개인정보보호에서 데이터 수집 원칙은 무엇인가요?",
        "AI 거버넌스 요구사항을 알려주세요."
    ]
    
    print("\n에이전트 테스트 시작")
    print("=" * 60)
    
    # 각 질문에 대해 테스트 수행
    for i, query in enumerate(test_queries, 1):
        print(f"\n[테스트 {i}/{len(test_queries)}]")
        test_agent_query(query)
        print("\n" + "="*60)
    
    print("\n모든 테스트 완료!")

# ==========================================
# 8. 대화형 모드 (선택적)
# ==========================================

def interactive_mode():
    """대화형 질의응답 모드"""
    print("\n대화형 모드 시작 (종료하려면 'quit' 입력)")
    print("-" * 50)
    
    while True:
        try:
            user_input = input("\n질문을 입력하세요: ").strip()
            
            if user_input.lower() in ['quit', 'exit', '종료']:
                print("대화를 종료합니다.")
                break
                
            if not user_input:
                continue
                
            test_agent_query(user_input)
            
        except KeyboardInterrupt:
            print("\n\n대화를 종료합니다.")
            break
        except Exception as e:
            print(f"오류 발생: {e}")

# 대화형 모드 실행 (필요시 주석 해제)
# interactive_mode()

llama-index 버전: 0.14.0
llama-index-core 버전: 0.14.1
--------------------------------------------------
LLM 및 임베딩 모델 설정 완료
데이터 디렉토리 및 샘플 데이터 준비 완료
문서 로딩 중...
로드된 문서 수: 2
벡터 인덱스 생성 중...


Generating embeddings: 100%|██████████| 2/2 [00:00<00:00,  5.93it/s]


벡터 인덱스 생성 완료
쿼리 엔진 및 검색 도구 생성 완료
ReAct 에이전트 생성 실패: from_tools
대안: 기본 쿼리 엔진을 직접 사용합니다.

에이전트 테스트 시작

[테스트 1/4]

질문: 2025년 AI 정책의 핵심 원칙은 무엇인가요?


2025-09-14 13:08:25,530 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-14 13:08:26,644 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



답변:
----------------------------------------
2025년 AI 정책의 핵심 원칙은 다음과 같습니다:
- 인간 중심의 AI 개발
- 투명성과 설명 가능성 보장
- 개인정보보호 및 프라이버시 강화
- 차별 방지 및 공정성 확보


[테스트 2/4]

질문: 차등 프라이버시에 대해 설명해주세요.


...


답변:
----------------------------------------
차등 프라이버시는 개인의 데이터가 포함된 데이터셋에서 특정 개인의 정보가 보호되도록 설계된 기술입니다. 이 방법은 데이터 분석 결과에 개인의 정보가 노출되지 않도록 하여, 데이터의 유용성을 유지하면서도 개인의 프라이버시를 강화하는 데 기여합니다. 이를 통해 데이터 수집 및 처리 과정에서 개인의 신원을 보호할 수 있습니다.


[테스트 3/4]

질문: 개인정보보호에서 데이터 수집 원칙은 무엇인가요?


2025-09-14 13:08:29,344 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-14 13:08:30,563 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



답변:
----------------------------------------
개인정보보호에서 데이터 수집 원칙은 다음과 같습니다:

1. 목적 제한 원칙: 명시된 목적으로만 수집
2. 최소 수집 원칙: 필요한 최소한의 데이터만 수집
3. 동의 기반 수집: 명확한 동의 절차 필수


[테스트 4/4]

질문: AI 거버넌스 요구사항을 알려주세요.


2025-09-14 13:08:30,901 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-14 13:08:32,673 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



답변:
----------------------------------------
AI 거버넌스 요구사항은 다음과 같습니다:

- AI 윤리 위원회 설치 의무
- 분기별 영향 평가 실시
- 시민 참여 메커니즘 구축


모든 테스트 완료!
